In [1]:
# Data downloaded on April 10, 2020
import pandas as pd
import numpy as np

In [2]:
bls_jobs_data = 'https://download.bls.gov/pub/time.series/ce/ce.data.0.AllCESSeries'

In [3]:
bls_df = pd.read_csv(bls_jobs_data,sep='\t')

In [4]:
bls_ind_code = 'https://download.bls.gov/pub/time.series/ce/ce.industry'
ind_code_cross = pd.read_csv(bls_ind_code, sep='\t')

In [5]:
ind_code_cross.columns = [x.strip() for x in ind_code_cross.columns]
ind_code_cross.naics_code = ind_code_cross.naics_code.str.strip()

ind_code_cross = ind_code_cross.loc[ind_code_cross.naics_code.str.len() == 3]
ind_code_cross.head()

,industry_code,naics_code,publishing_status,industry_name,display_level,selectable,sort_sequence
8,10211000,211,A,Oil and gas extraction,4,T,9
9,10212000,212,A,"Mining, except oil and gas",4,T,10
21,10213000,213,A,Support activities for mining,4,T,22
24,20236000,236,A,Construction of buildings,4,T,25
33,20237000,237,A,Heavy and civil engineering construction,4,T,34


In [6]:
bls_df = bls_df.loc[bls_df.year > 2018]
bls_df.columns = [x.strip() for x in bls_df.columns]

bls_df.series_id = bls_df.series_id.str.strip()
bls_df.period = bls_df.period.str.strip()
bls_df.footnote_codes = bls_df.footnote_codes.str.strip()

# Keep just all employment (data type = 01)
bls_df = bls_df.loc[bls_df.series_id.str[-2:] == '01']

# Keep only SA
bls_df = bls_df.loc[bls_df.series_id.str[0:3] == 'CES']

# Keep only 3-digit industry code
bls_df['industry_code'] = bls_df.series_id.str[3:11]
bls_df['industry_code'] = bls_df['industry_code'].astype(int)

bls_ind_df = pd.merge(bls_df,ind_code_cross,on='industry_code', how='right')
#bls_df.head()

bls_ind_df.set_index(['naics_code','year','period'],inplace=True)
bls_ind_df.head()

series_id  value footnote_codes  industry_code  \
naics_code year period                                                       
211        2019 M01     CES1021100001  142.4            NaN       10211000   
                M02     CES1021100001  143.6            NaN       10211000   
                M03     CES1021100001  143.9            NaN       10211000   
                M04     CES1021100001  146.1            NaN       10211000   
                M05     CES1021100001  147.8            NaN       10211000   

                       publishing_status           industry_name  \
naics_code year period                                             
211        2019 M01                    A  Oil and gas extraction   
                M02                    A  Oil and gas extraction   
                M03                    A  Oil and gas extraction   
                M04                    A  Oil and gas extraction   
                M05                    A  Oil and gas extraction   

                        display_level selectable  sort_sequence  
naics_code year period                                           
211        2019 M01                 4          T              9  
                M02                 4          T              9  
                M03                 4          T              9  
                M04                 4          T              9  
                M05                 4          T              9

In [7]:
# Log monthly difference
bls_ind_df['ln_emp'] = np.log(bls_ind_df.value)
bls_ind_df['ln_emp_diff'] = bls_ind_df.ln_emp.diff()

bls_ind_df.xs('722').tail()

series_id   value footnote_codes  industry_code  \
year period                                                        
2020 M04     CES7072200001  6227.1            NaN       70722000   
     M05     CES7072200001  7688.3            NaN       70722000   
     M06     CES7072200001  9156.2            NaN       70722000   
     M07     CES7072200001  9681.5              P       70722000   
     M08     CES7072200001  9815.1              P       70722000   

            publishing_status                      industry_name  \
year period                                                        
2020 M04                    A  Food services and drinking places   
     M05                    A  Food services and drinking places   
     M06                    A  Food services and drinking places   
     M07                    A  Food services and drinking places   
     M08                    A  Food services and drinking places   

             display_level selectable  sort_sequence    ln_emp  ln_emp_diff  
year period                                                                  
2020 M04                 4          T            808  8.736666    -0.628102  
     M05                 4          T            808  8.947455     0.210789  
     M06                 4          T            808  9.122187     0.174732  
     M07                 4          T            808  9.177972     0.055786  
     M08                 4          T            808  9.191677     0.013705

In [8]:
naics_wgts = bls_ind_df.xs([2020,'M03'],level=[1,2])[['ln_emp_diff']].reset_index()

naics_wgts['naics_code'] = naics_wgts['naics_code'].astype(int)
naics_wgts.head()

,naics_code,ln_emp_diff
0,211,-0.001278
1,212,-0.004783
2,213,-0.023811
3,236,-0.006653
4,237,-0.014205


In [9]:
naics_wgts.ln_emp_diff.describe()

count    76.000000
mean     -0.007758
std       0.012093
min      -0.052838
25%      -0.011650
50%      -0.004824
75%      -0.001284
max       0.017665
Name: ln_emp_diff, dtype: float64

In [10]:
# Get State Data from QCEW (Processed in QCEW Notebook)
df_state_ind_all = pd.read_csv('../../data/bls/state_emp_ind.csv')
df_state_ind_all.head()

df_state_ind_all['naics_code'] = df_state_ind_all['naics_code'].astype(int)

In [11]:
bls_state_ind_df = pd.merge(df_state_ind_all,naics_wgts,on='naics_code',how='right')

bls_state_ind_df['Total_Emp'] = bls_state_ind_df.groupby('area_fips')['annual_avg_emplvl'].transform('sum')

bls_state_ind_df['emp_wgt'] = bls_state_ind_df.annual_avg_emplvl/bls_state_ind_df.Total_Emp

bls_state_ind_df['emp_diff_times_share'] = bls_state_ind_df['emp_wgt']*bls_state_ind_df['ln_emp_diff']

bls_state_ind_df.head()

,area_fips,naics_code,annual_avg_emplvl,STATE,ln_emp_diff,Total_Emp,emp_wgt,emp_diff_times_share
0,1000,211,411,1.0,-0.001278,1492707,0.000275,-3.518706e-07
1,2000,211,3522,2.0,-0.001278,225543,0.015616,-1.995610e-05
2,4000,211,80,4.0,-0.001278,2168037,0.000037,-4.715622e-08
3,5000,211,675,5.0,-0.001278,922683,0.000732,-9.349039e-07
4,6000,211,4734,6.0,-0.001278,12694739,0.000373,-4.765629e-07


In [12]:
state_bartik_df = bls_state_ind_df.groupby('STATE')[['emp_diff_times_share']].sum().reset_index()

state_bartik_df.STATE = state_bartik_df.STATE.astype(int)
state_bartik_df.head()
state_bartik_df.to_csv('../../data/bls/state_bartiks.csv', index=False)